Contains code for halfspace on higher dims

In [1]:
import sys
sys.path.append('../../..')

import numpy as np
import random
import pandas as pd
import time
from matplotlib import pyplot as plt

from core.points import *
from core.ranges import *
from algorithms.epsnet import *
from core.ranges import get_range_space
from core.verification import is_epsnet

Sampling method:

In [2]:
def report_epsnet_sample(n, m, eps, dim, c1=1):
    # n = 2**10
    # m = 2**12
    # eps = 0.4
    success_prob = 0.9
    
    
    points = [Point(point=[random.uniform(0, 1), random.uniform(0, 1)], color=0) for _ in range(n // 2)]
    points += [Point(point=[random.uniform(0, 1), random.uniform(0, 1)], color=1) for _ in range(n // 2)]
    ranges = []
    for _ in range(m):
        # Generate a random normal vector in R^d
        normal = [random.uniform(-1, 1) for _ in range(dim)]
        
        # Calculate the range of possible dot products with the [0, 1]^d hypercube
        min_dot = sum(min(0, n) for n in normal)  # Minimum dot product with [0, 1]^d
        max_dot = sum(max(0, n) for n in normal)  # Maximum dot product with [0, 1]^d
        
        # Choose an offset within the range [min_dot, max_dot] to ensure intersection
        offset = random.uniform(min_dot, max_dot)
        
        # Create a HalfspaceRange object
        halfspace = HalfspaceRange(normal=normal, offset=offset)
        ranges.append(halfspace)
    
    rangespace = get_range_space(points, ranges)
    vc = dim + 1

    heavy_ranges = []
    for r in rangespace:
        if len(r) >= n * eps:
            heavy_ranges.append(r)
            
    print(f"n: {n}, m: {m}, vc: {vc}, epsilon: {eps}, number of heavy ranges: {len(heavy_ranges)}")

    # for _ in range(10):
    start = time.time()
    epsnet = build_epsnet_sample(
        points=points,
        rangespace=rangespace,
        vc=vc,
        epsilon=eps,
        success_prob=success_prob,
        weights=None,
        c1=c1
    )
    end = time.time()

    print(f"Number of points in epsnet: {len(epsnet)}, time taken: {end - start:.6f} seconds")

    success = is_epsnet(epsnet=epsnet, rangespace=rangespace, epsilon=eps)
    print(f"Success: {success}")  # Verify the eps-net
    
    return (n, m, 
            success_prob, eps, end - start, success, 
            len(heavy_ranges), 
            len([p for p in epsnet if p.color == 0]), len([p for p in epsnet if p.color == 1]))
    

In [29]:
n_values = [2**9, 2**10, 2**11, 2**12]
m_values = [2**9, 2**10, 2**11, 2**12]
dims = [4, 8, 16, 32]
eps_values = [0.9, 0.92, 0.94, 0.96, 0.98]
# c1 = dim / 4

aggregated_results = {
    "n": [],
    "m": [],
    "dim": [],
    "prob": [],
    "eps": [],
    "time": [],
    "success": [],
    "heavy_ranges": [],
    "tries": [],
    "red_points": [],
    "blue_points": []
}

for n in n_values:
    for m in m_values:
        for dim in dims:
            for eps in eps_values:
                tries = 10
                print(f"Running for n={n}, m={m}, dim={dim}, eps={eps}")
                result = report_epsnet_sample(n, m, eps, dim=dim, c1=dim//4)
                while not result[5] and tries > 0:
                    print("Retrying...")
                    result = report_epsnet_sample(n, m, eps, dim=dim, c1=dim//4)
                    tries -= 1
                
                aggregated_results["n"].append(result[0])
                aggregated_results["m"].append(result[1])
                aggregated_results["prob"].append(result[2])
                aggregated_results["dim"].append(dim)
                aggregated_results["eps"].append(result[3])
                aggregated_results["time"].append(result[4])
                aggregated_results["success"].append(result[5])
                aggregated_results["heavy_ranges"].append(result[6])
                aggregated_results["tries"].append(10 - tries)
                aggregated_results["red_points"].append(result[7])
                aggregated_results["blue_points"].append(result[8])

result = pd.DataFrame(aggregated_results)

Running for n=512, m=512, dim=4, eps=0.9
n: 512, m: 512, vc: 5, epsilon: 0.9, number of heavy ranges: 176
[build_epsnet_sample] epsnet size m: 185
Number of points in epsnet: 185, time taken: 0.000038 seconds
Success: True
Running for n=512, m=512, dim=4, eps=0.92
n: 512, m: 512, vc: 5, epsilon: 0.92, number of heavy ranges: 176
[build_epsnet_sample] epsnet size m: 180
Number of points in epsnet: 180, time taken: 0.000034 seconds
Success: True
Running for n=512, m=512, dim=4, eps=0.94
n: 512, m: 512, vc: 5, epsilon: 0.94, number of heavy ranges: 162
[build_epsnet_sample] epsnet size m: 175
Number of points in epsnet: 175, time taken: 0.000031 seconds
Success: True
Running for n=512, m=512, dim=4, eps=0.96
n: 512, m: 512, vc: 5, epsilon: 0.96, number of heavy ranges: 164
[build_epsnet_sample] epsnet size m: 170
Number of points in epsnet: 170, time taken: 0.000030 seconds
Success: True
Running for n=512, m=512, dim=4, eps=0.98
n: 512, m: 512, vc: 5, epsilon: 0.98, number of heavy ranges

In [ ]:
# result.to_csv("epsnet_sample_results_halfspace.csv", index=False)

Sketch and Merge:

In [3]:
def report_epsnet_sketch_merge(n, m, eps, dim, c1):
    success_prob = 0.9
    
    
    points = [Point(point=[random.uniform(0, 1), random.uniform(0, 1)], color=0) for _ in range(n // 2)]
    points += [Point(point=[random.uniform(0, 1), random.uniform(0, 1)], color=1) for _ in range(n // 2)]
    ranges = []
    for _ in range(m):
        # Generate a random normal vector in R^d
        normal = [random.uniform(-1, 1) for _ in range(dim)]
        
        # Calculate the range of possible dot products with the [0, 1]^d hypercube
        min_dot = sum(min(0, n) for n in normal)  # Minimum dot product with [0, 1]^d
        max_dot = sum(max(0, n) for n in normal)  # Maximum dot product with [0, 1]^d
        
        # Choose an offset within the range [min_dot, max_dot] to ensure intersection
        offset = random.uniform(min_dot, max_dot)
        
        # Create a HalfspaceRange object
        halfspace = HalfspaceRange(normal=normal, offset=offset)
        ranges.append(halfspace)
    
    rangespace = get_range_space(points, ranges)
    vc = dim + 1

    heavy_ranges = []
    for r in rangespace:
        if len(r) >= n * eps:
            heavy_ranges.append(r)
            
    print(f"n: {n}, m: {m}, vc: {vc}, epsilon: {eps}, number of heavy ranges: {len(heavy_ranges)}")
    
    # for _ in range(10):
    start = time.time()
    epsnet = build_epsnet_sketch_merge(
        points=points,
        rangespace=rangespace,
        vc=vc,
        epsilon=eps,
        c1=0,
        c2=c1
    )
    end = time.time()

    print(f"Number of points in epsnet: {len(epsnet)}, time taken: {end - start:.6f} seconds")

    success = is_epsnet(epsnet=epsnet, rangespace=rangespace, epsilon=eps)
    print(f"Success: {success}")  # Verify the eps-net
    
    return (n, m, 0.9, eps, end - start, success, len(heavy_ranges)
            , len([p for p in epsnet if p.color == 0]), len([p for p in epsnet if p.color == 1]))

In [7]:
n_values = [2**10]#, 2**12]
m_values = [2**9, 2**10, 2**11, 2**12]
dims = [4, 8, 16, 32]
eps_values = [0.9]
# c1 = dim / 4

aggregated_results = {
    "n": [],
    "m": [],
    "dim": [],
    "prob": [],
    "eps": [],
    "time": [],
    "success": [],
    "heavy_ranges": [],
    "tries": [],
    "red_points": [],
    "blue_points": []
}

for n in n_values:
    for m in m_values:
        for dim in dims:
            for eps in eps_values:
                tries = 10
                print(f"Running for n={n}, m={m}, dim={dim}, eps={eps}")
                result = report_epsnet_sketch_merge(n, m, eps, dim=dim, c1=dim//4)
                while not result[5] and tries > 0:
                    print("Retrying...")
                    result = report_epsnet_sketch_merge(n, m, eps, dim=dim, c1=dim//4)
                    tries -= 1
                
                aggregated_results["n"].append(result[0])
                aggregated_results["m"].append(result[1])
                aggregated_results["prob"].append(result[2])
                aggregated_results["dim"].append(dim)
                aggregated_results["eps"].append(result[3])
                aggregated_results["time"].append(result[4])
                aggregated_results["success"].append(result[5])
                aggregated_results["heavy_ranges"].append(result[6])
                aggregated_results["tries"].append(10 - tries)
                aggregated_results["red_points"].append(result[7])
                aggregated_results["blue_points"].append(result[8])

result = pd.DataFrame(aggregated_results)

Running for n=1024, m=512, dim=4, eps=0.9
n: 1024, m: 512, vc: 5, epsilon: 0.9, number of heavy ranges: 178
[build_epsnet_sketch_merge] epsnet size m: 185
[build_epsnet_sketch_merge] partition size p: 256
[build_epsnet_sketch_merge] Starting sketch-and-merge...
[_sketch_merge] pair: 1 / 2 of total nodes: 4
[_greedy_discrepancy_halving] counter: 256 / 256
[_sketch_merge] pair: 2 / 2 of total nodes: 4
[_greedy_discrepancy_halving] counter: 256 / 256
[_sketch_merge] pair: 1 / 1 of total nodes: 2
[_greedy_discrepancy_halving] counter: 256 / 256

Number of points in epsnet: 256, time taken: 30.012528 seconds
Success: True
Running for n=1024, m=512, dim=8, eps=0.9
n: 1024, m: 512, vc: 9, epsilon: 0.9, number of heavy ranges: 217
[build_epsnet_sketch_merge] epsnet size m: 167
[build_epsnet_sketch_merge] partition size p: 256
[build_epsnet_sketch_merge] Starting sketch-and-merge...
[_sketch_merge] pair: 1 / 2 of total nodes: 4
[_greedy_discrepancy_halving] counter: 256 / 256
[_sketch_merge] pa

In [ ]:
# result.to_csv("epsnet_sketch_merge_results_halfspace.csv", index=False)